# Main Results - Capacity

Developed by Siobhan Powell, 2021. Updated 2022.


In [1]:
import os
os.chdir('..')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import scipy
import os.path
import pandas
import matplotlib
import shutil
import copy

from simple_dispatch import generatorData
from simple_dispatch import bidStack
from simple_dispatch import dispatch
from simple_dispatch import generatorDataShort
from future_grid import FutureDemand
from future_grid import FutureGrid
from simple_dispatch import StorageModel

# Load Generator Model

In [4]:
gd_short = pickle.load(open('IntermediateOutputs/generator_data_short_WECC_2019.obj', 'rb'))

# Load 2035 Grid Model and Calculate Limits

In [4]:
grid = FutureGrid(gd_short)
grid.set_up_scenario(year=2035, solar=3.5, wind=3, fuel=1.0, ev_scenario='HighHome', ev_timers='_NoTimers', 
                     ev_pen=0.2, ev_workplace_control='avgem', ev_workplace_bool=False, evs_bool=True,
                     ev_scenario_date='20220313', weekend_date='20220313', weekend_timers='_NoTimers')
grid.check_overgeneration(save_str='testing')
grid.run_dispatch(0.1, save_str='Results/TestingLimits', return_generator_limits=True, thermal_storage=False, result_date='20220330')

limits_2035_long = copy.deepcopy(grid.dp.gen_limits.loc[:, ['datetime', 'limit2']])
limits_2035_long.to_csv('IntermediateOutputs/limits_2035_long.csv', index=None)



----Capacity too low----
Try with storage:
1.9% Complete
Limit value: 83187.57871232636
Limit value: 83187.57871232636
Limit value: 83187.57871232634
Limit value: 83187.57871232636
Limit value: 83187.57871232636
Limit value: 83187.57871232636
Limit value: 83187.57871232636
Limit value: 83187.57871232636
Limit value: 83187.57871232636
3.8% Complete
Limit value: 82784.66957343981
Limit value: 82784.66957343981
Limit value: 82784.66957343981
Limit value: 82784.66957343981
Limit value: 82784.66957343981
Limit value: 82784.66957343981
Limit value: 82784.66957343981
Limit value: 82784.66957343981
Limit value: 82784.66957343981
Limit value: 82784.66957343981
Limit value: 82784.66957343981
Limit value: 82784.66957343981
Limit value: 82784.66957343981
5.8% Complete
Limit value: 83611.8396133094
Limit value: 83611.83961330939
Limit value: 83611.83961330938
Limit value: 83611.83961330938
Limit value: 83611.83961330939
Limit value: 83611.83961330939
Limit value: 83611.83961330939
Limit value: 8361

42.3% Complete
Limit value: 78914.0227786734
Limit value: 78914.0227786734
Limit value: 78914.0227786734
Limit value: 78914.0227786734
Limit value: 78914.0227786734
Limit value: 78914.0227786734
Limit value: 78914.0227786734
Limit value: 78914.0227786734
Limit value: 78914.0227786734
Limit value: 78914.0227786734
Limit value: 78914.0227786734
44.2% Complete
Limit value: 82011.88438609574
Limit value: 82011.88438609574
Limit value: 82011.88438609574
Limit value: 82011.88438609574
Limit value: 82011.88438609574
Limit value: 82011.88438609574
Limit value: 82011.88438609574
Limit value: 82011.88438609574
Limit value: 82011.88438609574
46.2% Complete
Limit value: 82751.38946732659
Limit value: 82751.3894673266
Limit value: 82751.3894673266
Limit value: 82751.3894673266
Limit value: 82751.3894673266
Limit value: 82751.3894673266
Limit value: 82751.3894673266
Limit value: 82751.3894673266
Limit value: 82751.3894673266
48.1% Complete
Limit value: 81006.33783060905
Limit value: 81006.3378306090

Limit value: 82888.02818133609
Limit value: 82888.02818133609
Limit value: 82888.02818133609
Limit value: 82888.02818133609
Limit value: 82888.02818133609
Limit value: 82888.02818133609
Limit value: 82888.02818133609
Limit value: 82888.02818133609
Limit value: 82888.02818133609
Limit value: 82888.02818133609
Limit value: 82888.02818133609
Limit value: 82888.02818133609
Limit value: 82888.0281813361
94.2% Complete
Limit value: 82930.00492612908
Limit value: 82930.00492612908
Limit value: 82930.00492612908
Limit value: 82930.00492612908
Limit value: 82930.00492612908
Limit value: 82930.00492612908
Limit value: 82930.00492612908
Limit value: 82930.00492612908
Limit value: 82930.00492612908
96.2% Complete
Limit value: 82623.30091787408
Limit value: 82623.30091787406
Limit value: 82623.30091787406
Limit value: 82623.30091787406
Limit value: 82623.30091787406
Limit value: 82623.30091787406
Limit value: 82623.30091787406
Limit value: 82623.30091787406
Limit value: 82623.30091787406
Limit valu

In [5]:
limits_2035_long = pd.read_csv('IntermediateOutputs/limits_2035_long.csv')

# with 1, 5, 10 GW storage

In [5]:

timer_names = {'':'Timers9pm', '_midnighttimers':'Timers12am', '_NoTimers':'TimersNone', '_RandomTimers':'RandomTimers'}
maxpen_results1 = {}
maxpen_results5 = {}
maxpen_results10 = {}

fuel = 1
solar = 3.5
wind = 3
for ev_scenario in ['UniversalHome', 'HighHome', 'LowHome_HighWork', 'LowHome_LowWork']:
    # no workplace control
    ev_workplace_bool=False
    results1 = np.zeros((4,))
    results5 = np.zeros((4,))
    results10 = np.zeros((4,))
    for i, ev_timers in enumerate(['_RandomTimers', '', '_midnighttimer', '_NoTimers']):
        ev_scenario_date = '20220313'
        grid = FutureGrid(gd_short)
        max_penlevel1 = grid.find_capacity_limit_1_binarysearch(with_storage_before=True, cap=4000, max_rate=1000, bs_limits=None, lims_8760=limits_2035_long['limit2'].values, 
                                                                year=2035, solar=solar, wind=wind, fuel=fuel,
                                                                ev_scenario=ev_scenario, ev_timers=ev_timers, ev_workplace_control='', ev_workplace_bool=ev_workplace_bool, evs_bool=True, ev_scenario_date=ev_scenario_date)
        results1[i] = max_penlevel1
        max_penlevel5 = grid.find_capacity_limit_1_binarysearch(with_storage_before=True, cap=20000, max_rate=5000, bs_limits=None, lims_8760=limits_2035_long['limit2'].values, 
                                                                year=2035, solar=solar, wind=wind, fuel=fuel,
                                                                ev_scenario=ev_scenario, ev_timers=ev_timers, ev_workplace_control='', ev_workplace_bool=ev_workplace_bool, evs_bool=True, ev_scenario_date=ev_scenario_date)
        results5[i] = max_penlevel5
        max_penlevel10 = grid.find_capacity_limit_1_binarysearch(with_storage_before=True, cap=40000, max_rate=10000, bs_limits=None, lims_8760=limits_2035_long['limit2'].values, 
                                                                year=2035, solar=solar, wind=wind, fuel=fuel,
                                                                ev_scenario=ev_scenario, ev_timers=ev_timers, ev_workplace_control='', ev_workplace_bool=ev_workplace_bool, evs_bool=True, ev_scenario_date=ev_scenario_date)
        results10[i] = max_penlevel10
    print('fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_noWPcontrol')
    maxpen_results1['fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_noWPcontrol'] = results1
    maxpen_results5['fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_noWPcontrol'] = results5
    maxpen_results10['fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_noWPcontrol'] = results10

    # workplace control
    ev_workplace_bool=True
    for wp_control in ['avgem', 'minpeak']:
        results1 = np.zeros((4,))
        results5 = np.zeros((4,))
        results10 = np.zeros((4,))
        for i in range(3):
            results1[i] = np.nan
            results5[i] = np.nan
            results10[i] = np.nan

        i = 3
        ev_timers = '_NoTimers'
        ev_scenario_date = '20220313'
        grid = FutureGrid(gd_short)
        max_penlevel1 = grid.find_capacity_limit_1_binarysearch(with_storage_before=True, cap=4000, max_rate=1000, bs_limits=None, lims_8760=limits_2035_long['limit2'].values, 
                                                                year=2035, solar=solar, wind=wind, fuel=fuel,
                                                                ev_scenario=ev_scenario, ev_timers=ev_timers, ev_workplace_control='', ev_workplace_bool=ev_workplace_bool, evs_bool=True, ev_scenario_date=ev_scenario_date)
        results1[i] = max_penlevel1
        max_penlevel5 = grid.find_capacity_limit_1_binarysearch(with_storage_before=True, cap=20000, max_rate=5000, bs_limits=None, lims_8760=limits_2035_long['limit2'].values, 
                                                                year=2035, solar=solar, wind=wind, fuel=fuel,
                                                                ev_scenario=ev_scenario, ev_timers=ev_timers, ev_workplace_control='', ev_workplace_bool=ev_workplace_bool, evs_bool=True, ev_scenario_date=ev_scenario_date)
        results5[i] = max_penlevel5
        max_penlevel10 = grid.find_capacity_limit_1_binarysearch(with_storage_before=True, cap=40000, max_rate=10000, bs_limits=None, lims_8760=limits_2035_long['limit2'].values, 
                                                                year=2035, solar=solar, wind=wind, fuel=fuel,
                                                                ev_scenario=ev_scenario, ev_timers=ev_timers, ev_workplace_control='', ev_workplace_bool=ev_workplace_bool, evs_bool=True, ev_scenario_date=ev_scenario_date)
        results10[i] = max_penlevel10
        print('fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_WPcontrol_'+wp_control)
        maxpen_results1['fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_WPcontrol_'+wp_control] = results1
        maxpen_results5['fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_WPcontrol_'+wp_control] = results5
        maxpen_results10['fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_WPcontrol_'+wp_control] = results10

maxpen_results1GW = pd.DataFrame(maxpen_results1, index=['TimersRandom', 'Timers9pm', 'Timers12am', 'TimersNone'])
maxpen_results1GW.to_csv('Results/max_penetration_levels_1h_2035_1GWstorage_20220408.csv')
maxpen_results5GW = pd.DataFrame(maxpen_results5, index=['TimersRandom', 'Timers9pm', 'Timers12am', 'TimersNone'])
maxpen_results5GW.to_csv('Results/max_penetration_levels_1h_2035_5GWstorage_20220408.csv')
maxpen_results10GW = pd.DataFrame(maxpen_results10, index=['TimersRandom', 'Timers9pm', 'Timers12am', 'TimersNone'])
maxpen_results10GW.to_csv('Results/max_penetration_levels_1h_2035_10GWstorage_20220408.csv')

Short Binary Search: 
Solving optimization.
0.5 : 16
Solving optimization.
0.26 : 5
Solving optimization.
0.14 : 1
Linear search from starting point:  0.01
0.01
Solving optimization.
0.02
Solving optimization.
0.03
Solving optimization.
0.04
Solving optimization.
0.05
Solving optimization.
0.060000000000000005
Solving optimization.
0.06999999999999999
Solving optimization.
Total overs:  1
Violation 1:  0.06999999999999999
Short Binary Search: 
Solving optimization.
0.5 : 3
Solving optimization.
0.26 : 0
Solving optimization.
0.38 : 0
Linear search from starting point:  0.38
0.38
Solving optimization.
0.39
Solving optimization.
0.4
Solving optimization.
0.41000000000000003
Solving optimization.
0.42000000000000004
Solving optimization.
0.43000000000000005
Solving optimization.
0.44000000000000006
Solving optimization.
0.45000000000000007
Solving optimization.
0.4600000000000001
Solving optimization.
Total overs:  1
Violation 1:  0.4600000000000001
Short Binary Search: 
Solving optimizat

Solving optimization.
0.3800000000000001
Solving optimization.
Total overs:  2
Violation 1:  0.3800000000000001
Short Binary Search: 
Solving optimization.
0.5 : 0
Solving optimization.
0.75 : 4
Solving optimization.
0.62 : 0
Linear search from starting point:  0.62
0.62
Solving optimization.
0.63
Solving optimization.
0.64
Solving optimization.
0.65
Solving optimization.
0.66
Solving optimization.
0.67
Solving optimization.
Total overs:  1
Violation 1:  0.67
fuel1_solar3.5_wind3_UniversalHome_WPcontrol_minpeak
Short Binary Search: 
Solving optimization.
0.5 : 8
Solving optimization.
0.26 : 4
Solving optimization.
0.14 : 1
Linear search from starting point:  0.01
0.01
Solving optimization.
0.02
Solving optimization.
0.03
Solving optimization.
0.04
Solving optimization.
0.05
Solving optimization.
0.060000000000000005
Solving optimization.
0.06999999999999999
Solving optimization.
0.08
Solving optimization.
Total overs:  1
Violation 1:  0.08
Short Binary Search: 
Solving optimization.
0.

Solving optimization.
0.5 : 6
Solving optimization.
0.26 : 1
Solving optimization.
0.14 : 1
Linear search from starting point:  0.01
0.01
Solving optimization.
0.02
Solving optimization.
0.03
Solving optimization.
0.04
Solving optimization.
0.05
Solving optimization.
0.060000000000000005
Solving optimization.
0.06999999999999999
Solving optimization.
0.08
Solving optimization.
0.09
Solving optimization.
Total overs:  1
Violation 1:  0.09
Short Binary Search: 
Solving optimization.
0.5 : 0
Solving optimization.
0.75 : 1
Solving optimization.
0.62 : 0
Linear search from starting point:  0.62
0.62
Solving optimization.
0.63
Solving optimization.
0.64
Solving optimization.
0.65
Solving optimization.
0.66
Solving optimization.
Total overs:  1
Violation 1:  0.66
Short Binary Search: 
Solving optimization.
0.5 : 0
Solving optimization.
0.75 : 0
Solving optimization.
0.88 : 0
Linear search from starting point:  0.88
0.88
Solving optimization.
0.89
Solving optimization.
0.9
Solving optimization

0.56
Solving optimization.
0.5700000000000001
Solving optimization.
0.5800000000000001
Solving optimization.
0.5900000000000001
Solving optimization.
0.6000000000000001
Solving optimization.
0.6100000000000001
Solving optimization.
Total overs:  1
Violation 1:  0.6100000000000001
Short Binary Search: 
Solving optimization.
0.5 : 0
Solving optimization.
0.75 : 0
Solving optimization.
0.88 : 0
Linear search from starting point:  0.88
0.88
Solving optimization.
0.89
Solving optimization.
0.9
Solving optimization.
0.91
Solving optimization.
0.92
Solving optimization.
0.93
Solving optimization.
0.9400000000000001
Solving optimization.
0.9500000000000001
Solving optimization.
0.9600000000000001
Solving optimization.
0.9700000000000001
Solving optimization.
0.9800000000000001
Solving optimization.
0.9900000000000001
Solving optimization.
1.0
Solving optimization.
fuel1_solar3.5_wind3_LowHome_HighWork_WPcontrol_minpeak
Short Binary Search: 
Solving optimization.
0.5 : 6
Solving optimization.
0

0.03
Solving optimization.
0.04
Solving optimization.
0.05
Solving optimization.
0.060000000000000005
Solving optimization.
0.06999999999999999
Solving optimization.
0.08
Solving optimization.
Total overs:  1
Violation 1:  0.08
Short Binary Search: 
Solving optimization.
0.5 : 0
Solving optimization.
0.75 : 2
Solving optimization.
0.62 : 1
Linear search from starting point:  0.5
0.5
Solving optimization.
0.51
Solving optimization.
0.52
Solving optimization.
0.53
Solving optimization.
0.54
Solving optimization.
0.55
Solving optimization.
0.56
Solving optimization.
0.5700000000000001
Solving optimization.
0.5800000000000001
Solving optimization.
Total overs:  1
Violation 1:  0.5800000000000001
Short Binary Search: 
Solving optimization.
0.5 : 0
Solving optimization.
0.75 : 0
Solving optimization.
0.88 : 0
Linear search from starting point:  0.88
0.88
Solving optimization.
0.89
Solving optimization.
0.9
Solving optimization.
0.91
Solving optimization.
0.92
Solving optimization.
0.93
Solvi

# Business As Usual Case

In [ ]:
timer_names = {'':'Timers9pm', '_midnighttimers':'Timers12am', '_NoTimers':'TimersNone'}
maxpen_results1 = {}
maxpen_results5 = {}
maxpen_results10 = {}

fuel = 1
ev_scenario = 'BusinessAsUsual'
ev_workplace_bool = True
ev_timers = ''
wp_control = 'minpeak'
ev_scenario_date = '20220313'

solar = 3.5
wind = 3

grid = FutureGrid(gd_short)
max_penlevel1 = grid.find_capacity_limit_1_binarysearch(with_storage_before=True, cap=4000, max_rate=1000, bs_limits=None, lims_8760=limits_2035_long['limit2'].values, 
                                                        year=2035, solar=solar, wind=wind, fuel=fuel,
                                                        ev_scenario=ev_scenario, ev_timers=ev_timers, ev_workplace_control='', ev_workplace_bool=ev_workplace_bool, evs_bool=True, ev_scenario_date=ev_scenario_date)
results1[i] = max_penlevel1
max_penlevel5 = grid.find_capacity_limit_1_binarysearch(with_storage_before=True, cap=20000, max_rate=5000, bs_limits=None, lims_8760=limits_2035_long['limit2'].values, 
                                                        year=2035, solar=solar, wind=wind, fuel=fuel,
                                                        ev_scenario=ev_scenario, ev_timers=ev_timers, ev_workplace_control='', ev_workplace_bool=ev_workplace_bool, evs_bool=True, ev_scenario_date=ev_scenario_date)
results5[i] = max_penlevel5
max_penlevel10 = grid.find_capacity_limit_1_binarysearch(with_storage_before=True, cap=40000, max_rate=10000, bs_limits=None, lims_8760=limits_2035_long['limit2'].values, 
                                                        year=2035, solar=solar, wind=wind, fuel=fuel,
                                                        ev_scenario=ev_scenario, ev_timers=ev_timers, ev_workplace_control='', ev_workplace_bool=ev_workplace_bool, evs_bool=True, ev_scenario_date=ev_scenario_date)
results10[i] = max_penlevel10
print('fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_WPcontrol_'+wp_control)

maxpen_results1['fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_WPcontrol_'+wp_control] = max_penlevel1
maxpen_results5['fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_WPcontrol_'+wp_control] = max_penlevel5
maxpen_results10['fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_WPcontrol_'+wp_control] = max_penlevel10

maxpen_results1GW = pd.DataFrame(maxpen_results1, index=['TimersMixed'])
maxpen_results1GW.to_csv('Results/max_penetration_levels_1h_bau_2035_1GWstorage_20220408.csv')
maxpen_results5GW = pd.DataFrame(maxpen_results5, index=['TimersMixed'])
maxpen_results5GW.to_csv('Results/max_penetration_levels_1h_bau_2035_5GWstorage_20220408.csv')
maxpen_results10GW = pd.DataFrame(maxpen_results10, index=['TimersMixed'])
maxpen_results10GW.to_csv('Results/max_penetration_levels_1h_bau_2035_10GWstorage_20220408.csv')